In [1]:
import sys
import spikeinterface as si
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import probeinterface

from probeinterface import Probe, ProbeGroup

import os
import numpy as np
from spikeinterface.core import concatenate_recordings

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from matplotlib.collections import LineCollection
from probeinterface import write_probeinterface, read_probeinterface


/home/ubuntu/.conda/envs/spike_sorting/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_list = os.listdir(f"/media/ubuntu/sda/mouse_test/raw_data/20250914_Janus1_2_250914_134141")
file_list.remove("settings.xml")
recording_raw_list = []
for file in file_list:
    recording_raw_list.append(se.read_intan(f"/media/ubuntu/sda/mouse_test/raw_data/20250914_Janus1_2_250914_134141/{file}", stream_id= '0'))
recording_raw = concatenate_recordings(recording_list=recording_raw_list)

recording_raw = spre.unsigned_to_signed(recording_raw)
recording_recorded = spre.bandpass_filter(recording_raw, freq_min=300, freq_max=3000)
recording_recorded = spre.notch_filter(recording_recorded, freq=50)
recording_f = spre.common_reference(recording_recorded, reference="global", operator="median")

In [4]:
probe_30channel = read_probeinterface('/media/ubuntu/sda/mouse_test/probe/tip_probe.json')
recording_f = recording_f.set_probegroup(probe_30channel)

In [6]:
output_folder = '/media/ubuntu/sda/mouse_test/sorted/20250914_Janus1_2_250914_134141'
recording_preprocessed = recording_f.save(format="binary")
print(recording_preprocessed)

sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, folder=output_folder + "/kilosort4")
analyzer_kilosort4 = si.create_sorting_analyzer(sorting=sorting_kilosort4, recording=recording_preprocessed, format='binary_folder', folder=output_folder + '/analyzer_kilosort4_binary')

extensions_to_compute = [
    "random_spikes",
    "waveforms",
    "noise_levels",
    "templates",
    "spike_amplitudes",
    "unit_locations",
    "spike_locations",
    "correlograms",
    "template_similarity"
]

extension_params = {
    "unit_locations": {"method": "center_of_mass"},
    "spike_locations": {"ms_before": 0.1},
    "correlograms": {"bin_ms": 0.1},
    "template_similarity": {"method": "cosine_similarity"}
}

analyzer_kilosort4.compute(extensions_to_compute, extension_params=extension_params)

qm_params = sqm.get_default_qm_params()
analyzer_kilosort4.compute("quality_metrics", qm_params)

import spikeinterface.exporters as sexp
sexp.export_to_phy(analyzer_kilosort4, output_folder + "/phy_folder_for_kilosort", verbose=True)

Use cache_folder=/tmp/spikeinterface_cache/tmpwk2hwdgy/BE7QDE9U
write_binary_recording 
engine=process - n_jobs=1 - samples_per_chunk=20,000 - chunk_memory=1.14 MiB - total_memory=1.14 MiB - chunk_duration=1.00s


write_binary_recording (no parallelization): 100%|██████████| 422/422 [00:09<00:00, 46.76it/s]


BinaryFolderRecording: 30 channels - 20.0kHz - 1 segments - 8,438,144 samples 
                       421.91s (7.03 minutes) - int16 dtype - 482.83 MiB


noise_level (no parallelization): 100%|██████████| 20/20 [00:00<00:00, 211.49it/s]
Compute : spike_amplitudes + spike_locations (no parallelization): 100%|██████████| 422/422 [00:00<00:00, 622.56it/s]
extract PCs (no parallelization): 100%|██████████| 422/422 [00:18<00:00, 23.08it/s]

Run:
phy template-gui  /media/ubuntu/sda/mouse_test/sorted/20250914_Janus1_2_250914_134141/phy_folder_for_kilosort/params.py
